In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

llm = ChatOpenAI(
    model="ai/smollm3",
    base_url="http://127.0.0.1:12434/engines/v1",
    api_key="ignored"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant! Your name is Bob."),
    MessagesPlaceholder("history"),
    ("human", "{input}")
])


chain = prompt | llm
store = {}
sessionId = 'abc123'

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]
    
runnable_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"    
)

while True:
    user_input = input("Enter your message (type 'exit' to end): ")
    if user_input.lower() == "exit":
        break
    output = runnable_message_history.invoke(
        { 
            # "history": [("human", "what is the value of 5 * 5?"), ("ai", "The value of 5 * 5 is 25.")], 
            "input": user_input
        },
        config={"configurable": {"session_id": sessionId}});
    print(output.content)
